In [ ]:
import torch
from transformers import pipeline

In [ ]:
MODEL_NAME = "joeddav/distilbert-base-uncased-go-emotions-student"

PHRASES = [
  "What a wonderful day",
  "OMG my head hurts",
  "What am I doing here?"
]

In [ ]:
def sort_by_score(scores):
  return sorted(scores, key=lambda A: A["score"], reverse=True)

pipe = pipeline(
  "sentiment-analysis",
  model=MODEL_NAME,
  torch_dtype=torch.bfloat16,
  return_all_scores=True
)


for p in PHRASES:
  print(p)
  scores = pipe(p)[0]
  display(sort_by_score(scores)[:3])